In [531]:
import pandas as pd
import numpy as np

In [532]:
### DEFINE BETA VALUES FOR VDF

beta_vals = {} # TBD: compute values based on growth

beta_template_aux = {
    "Period": ["AM", "MD", "PM", "NT"],
    1: [3.246, 3.249, 3.399, 3.402],
    2: [0.715, 0.708, 0.592, 0.708],
    3: [3.130, 3.133, 3.283, 3.286]
}

beta_vals_aux = pd.DataFrame(beta_template_aux)

beta_vals_aux.set_index("Period", inplace=True)

beta_vals[2025] = beta_vals_aux

In [533]:
### DEFINE LOOKUP TABLE FOR BONUS PER PERIOD

lookup_period_file = r"inputs/LookUp_Period.csv"

lookup_period = pd.read_csv(
    lookup_period_file,
    sep=",",          # `delimiter` y `sep` son equivalentes; elige uno
    encoding="utf-8",
    decimal=".",      # parsea decimales con punto
    thousands=",",    # parsea separador de miles con coma
    quotechar='"',
    index_col=0
)

# Clip y reasignar
lookup_period = lookup_period*0

lookup_period

,Bonus/Mile,4 Periods
Period,,
Night,0.0,
AM-Early,0.0,
AM-Peak,0.0,
AM-Shoulder,0.0,
MD,0.0,
PM-Shoulder,0.0,
PM-Peak,0.0,
PM-Late,0.0,


In [534]:
### DEFINE SEGMENT PARAMETERS
# Default configuration for time periods in traffic data

#TBD: Make this automatically
period_template = [                 # (Period, Hours/Day, Peak/OP, 4Periods tag)
    ("Night",        7, "OP",   "NT"),
    ("AM-Early",     1, "OP",   "AM"),
    ("AM-Peak",      3, "Peak", "AM"),
    ("AM-Shoulder",  1, "OP",   "AM"),
    ("MD",           5, "OP",   "MD"),
    ("PM-Shoulder",  1, "OP",   "PM"),
    ("PM-Peak",      3, "Peak", "PM"),
    ("PM-Late",      3, "OP",   "PM"),
]

rows = []
years = [2025]

# Default time periods list (for reference)
default_time_periods = [
    "Night",
    "AM-Early",
    "AM-Peak",
    "AM-Shoulder",
    "MD",
    "PM-Shoulder",
    "PM-Peak",
    "PM-Late"
]

# Create the base scenario: hour -> time period mapping
hour_to_period = {
    0: "Night",
    1: "Night",
    2: "Night",
    3: "Night",
    4: "Night",
    5: "Night",
    6: "AM-Early",
    7: "AM-Peak",
    8: "AM-Peak",
    9: "AM-Peak",
    10: "AM-Shoulder",
    11: "MD",
    12: "MD",
    13: "MD",
    14: "MD",
    15: "MD",
    16: "PM-Shoulder",
    17: "PM-Peak",
    18: "PM-Peak",
    19: "PM-Peak",
    20: "PM-Late",
    21: "PM-Late",
    22: "PM-Late",
    23: "Night"
}

period_to_period = {
    'Evening': 'PM-Late',
    'Evening': 'Night',
    'EarlyAM': 'AM-Early',
    'AM': 'AM-Peak',
    'AM': 'AM-Shoulder',
    'Midday': 'MD',
    'PM': 'PM-Shoulder',
    'PM': 'PM-Peak'
}

# Define the segments and their parameters

peak_factor = 1 # 1.05 # Peak factor for adjustment at peak hour traffic

hov_percentage = pd.DataFrame({
    'Year' : [2025],
    'HOV percentage' : [0]
})

hov_percentage.set_index('Year', inplace=True)

# Define segment parameters base
seg_params = pd.DataFrame({
    'SegDir':   ["1NB","1SB","2NB","2SB","3NB","3SB","4NB","4SB","5NB","5SB","6NB","6SB","7NB","7SB","8NB","8SB","9NB","9SB","10NB","10SB"],
    'Length':    [1.3,1.3,1.3,1.3,0.5,0.5,1.6,1.6,2,2,3.6,3.6,2.9,2.9,3.8,3.8,3.4,3.4,4.5,4.5],
    'Inscope':   [0.8]*20,
    'Lanes_GP':  [4]*20, #
    'Lanes_ML':  [2]*20, # Lanes_ML': [2,2,2,2,2,2,2,2,3,3,2,2,2,2], # Do test changing segment 5
    'CapPerLane_GP': [2000]*20,
    'CapPerLane_ML': [1800]*20,
    'Speed_GP':  [55]*6 + [65]*2 + [70]*12,
    'Speed_ML':  [70]*20,
    'Alpha_GP':  [1]*20,
    'Beta_GP':   [6]*20,
    'Alpha_ML':  [1.6]*20,
    'Beta_ML':   [6.3]*20,
    'Min_Toll_2016': [None]*20,
    'Max_Toll_2016': [None]*20,
    'LanesGP_AM_Peak': [5]*20,
    'LanesGP_PM_Peak': [5]*20,
})

seg_params.set_index('SegDir', inplace=True)

# Compute capacities as lanes * cap per lane
seg_params['Cap_GP'] = seg_params['Lanes_GP'] * seg_params['CapPerLane_GP']
seg_params['Cap_ML'] = seg_params['Lanes_ML'] * seg_params['CapPerLane_ML']

# Compute peak capacities as Alpha * base capacity
seg_params['CapGP_Peak'] = seg_params['Alpha_GP'] * seg_params['Cap_GP']
seg_params['CapML_Peak'] = seg_params['Alpha_ML'] * seg_params['Cap_ML']

# Optional: if you want integer capacities
seg_params[['Cap_GP','Cap_ML','CapGP_Peak','CapML_Peak']] = seg_params[
    ['Cap_GP','Cap_ML','CapGP_Peak','CapML_Peak']
].astype(int)

# Preview
seg_params

,Length,Inscope,Lanes_GP,Lanes_ML,CapPerLane_GP,CapPerLane_ML,Speed_GP,Speed_ML,Alpha_GP,Beta_GP,Alpha_ML,Beta_ML,Min_Toll_2016,Max_Toll_2016,LanesGP_AM_Peak,LanesGP_PM_Peak,Cap_GP,Cap_ML,CapGP_Peak,CapML_Peak
SegDir,,,,,,,,,,,,,,,,,,,,
1NB,1.3,0.8,4,2,2000,1800,55,70,1,6,1.6,6.3,None,None,5,5,8000,3600,8000,5760
1SB,1.3,0.8,4,2,2000,1800,55,70,1,6,1.6,6.3,None,None,5,5,8000,3600,8000,5760
2NB,1.3,0.8,4,2,2000,1800,55,70,1,6,1.6,6.3,None,None,5,5,8000,3600,8000,5760
2SB,1.3,0.8,4,2,2000,1800,55,70,1,6,1.6,6.3,None,None,5,5,8000,3600,8000,5760
3NB,0.5,0.8,4,2,2000,1800,55,70,1,6,1.6,6.3,None,None,5,5,8000,3600,8000,5760
3SB,0.5,0.8,4,2,2000,1800,55,70,1,6,1.6,6.3,None,None,5,5,8000,3600,8000,5760
4NB,1.6,0.8,4,2,2000,1800,65,70,1,6,1.6,6.3,None,None,5,5,8000,3600,8000,5760
4SB,1.6,0.8,4,2,2000,1800,65,70,1,6,1.6,6.3,None,None,5,5,8000,3600,8000,5760
5NB,2.0,0.8,4,2,2000,1800,70,70,1,6,1.6,6.3,None,None,5,5,8000,3600,8000,5760


In [535]:
import numpy as np

def adjusted_cumprod(row, target_year, multiplier):
    years = row.index
    print(row.values)
    factors = 1 + row.values
    
    # Find the index of the target year
    target_idx = list(years).index(target_year)
    
    # Apply multiplier to the target year's factor
    factors[target_idx] *= multiplier
    
    # Calculate cumulative product
    return pd.Series(np.cumprod(factors), index=years)

In [536]:
### HERE ARE THE MEASURED SPEEDS WE USE AS REFERENCE

measured_speeds_file = r"inputs/measured_speeds.csv"

measured_speeds_aux = pd.read_csv(
    measured_speeds_file,
    sep=",",          # `delimiter` y `sep` son equivalentes; elige uno
    encoding="utf-8",
    decimal=".",      # parsea decimales con punto
    thousands=",",    # parsea separador de miles con coma
    quotechar='"',
    index_col=0
)

# Convert the dictionary into a Series indexed like the rows of your df
period_index = pd.Series(hour_to_period)

# Assign period names to each row using the hour index
df_period = measured_speeds_aux.copy()
df_period["Period"] = df_period.index.map(period_index)

# Group by period and average
measured_speeds = df_period.groupby("Period").mean()

measured_speeds

,1NB,2NB,3NB,4NB,5NB,6NB,7NB,8NB,9NB,10NB,1SB,2SB,3SB,4SB,5SB,6SB,7SB,8SB,9SB,10SB
Period,,,,,,,,,,,,,,,,,,,,
AM-Early,53.492043,66.842800,53.889918,63.262233,63.937581,61.139591,61.863397,63.539264,64.286733,69.142619,59.226645,67.331894,64.249888,60.360098,66.965849,66.681046,65.483755,67.618570,68.651330,69.707134
AM-Peak,23.368767,16.611981,29.311822,36.694192,32.846491,26.733634,17.495310,22.799974,60.771390,67.573757,55.165686,66.596026,14.980211,58.114067,63.530433,63.509193,63.354576,64.611111,67.098144,68.664265
AM-Shoulder,22.128004,16.998816,22.799581,42.927944,40.179354,37.906546,38.152687,53.763993,64.939390,66.938870,54.164539,66.228092,16.288704,56.587592,62.425792,63.490565,63.591161,65.090773,66.517092,67.739594
MD,27.969299,40.336862,39.298965,62.404440,54.923844,58.801746,61.912022,63.833117,65.139533,66.994121,42.310764,57.523702,21.431522,48.719222,51.471449,61.269985,63.174072,64.977579,67.214343,69.024712
Night,59.403833,65.426109,65.865456,65.234369,66.781890,61.237078,59.891782,64.193947,65.022995,66.400907,58.406691,66.706496,61.351397,58.549796,64.702160,63.593358,63.867150,65.986152,66.577333,67.675741
PM-Late,53.298340,65.010407,62.398853,65.172424,67.048250,61.804450,60.079948,62.684476,64.087294,66.597165,50.071985,65.515962,59.853843,46.789705,49.197876,51.638709,59.778301,64.160576,66.868372,68.482259
PM-Peak,19.131858,40.659541,25.491779,54.003463,60.659572,58.734378,59.111685,56.346053,62.414082,64.964063,18.742579,29.700200,20.762388,9.845916,20.139612,27.724473,38.275798,50.970429,61.532810,65.411155
PM-Shoulder,20.169131,37.920435,22.799581,60.099121,63.937581,61.139591,61.863397,62.127280,63.331980,65.888848,21.265943,26.233205,21.416629,13.074389,26.982577,41.859710,50.697101,50.837319,54.397452,67.197678


In [537]:
### IMPORT GROWTHS FOR EACH CLASS
file_path_growths = r"inputs/growths_per_segment.csv"
base_growth_df = pd.read_csv(
    file_path_growths,
    delimiter=',',
    encoding='utf-8',
    decimal='.',        # ← this tells pandas how to parse decimals
    thousands=',',       # ← this tells pandas how to parse thousands
    quotechar='"'
)

base_growth_df = base_growth_df.iloc[:, 1:]
project_years = base_growth_df.columns[1:].tolist()
base_growth_df.iloc[:, 1:] =  base_growth_df.iloc[:, 1:] + 1

base_growth_df.loc[:, '2032'] *= 1.12

base_growth_df

,SegmentMapped,2026,2027,2028,2029,2030,2031,2032,2033,2034,...,2062,2063,2064,2065,2066,2067,2068,2069,2070,2071
0,S1,1.018928,1.018928,1.018928,1.018928,1.018928,1.022899,1.145647,1.022899,1.022899,...,1.014078,1.014078,1.014078,1.014078,1.008447,1.008447,1.008447,1.008447,1.008447,1.008447
1,S2,1.025583,1.025583,1.025583,1.025583,1.025583,1.031514,1.155296,1.031514,1.031514,...,1.016660,1.016660,1.016660,1.016660,1.009996,1.009996,1.009996,1.009996,1.009996,1.009996
2,S3,1.023025,1.023025,1.023025,1.023025,1.023025,1.028939,1.152412,1.028939,1.028939,...,1.015544,1.015544,1.015544,1.015544,1.009326,1.009326,1.009326,1.009326,1.009326,1.009326
3,S4,1.022150,1.022150,1.022150,1.022150,1.022150,1.027914,1.151263,1.027914,1.027914,...,1.015180,1.015180,1.015180,1.015180,1.009108,1.009108,1.009108,1.009108,1.009108,1.009108
4,S5,1.018077,1.018077,1.018077,1.018077,1.018077,1.022963,1.145719,1.022963,1.022963,...,1.013537,1.013537,1.013537,1.013537,1.008122,1.008122,1.008122,1.008122,1.008122,1.008122
5,S6,1.018314,1.018314,1.018314,1.018314,1.018314,1.023435,1.146247,1.023435,1.023435,...,1.013597,1.013597,1.013597,1.013597,1.008158,1.008158,1.008158,1.008158,1.008158,1.008158
6,S7,1.020046,1.020046,1.020046,1.020046,1.020046,1.025430,1.148481,1.025430,1.025430,...,1.014297,1.014297,1.014297,1.014297,1.008578,1.008578,1.008578,1.008578,1.008578,1.008578
7,S8,1.019537,1.019537,1.019537,1.019537,1.019537,1.025049,1.148055,1.025049,1.025049,...,1.014060,1.014060,1.014060,1.014060,1.008436,1.008436,1.008436,1.008436,1.008436,1.008436
8,S9,1.017924,1.017924,1.017924,1.017924,1.017924,1.022940,1.145693,1.022940,1.022940,...,1.013443,1.013443,1.013443,1.013443,1.008066,1.008066,1.008066,1.008066,1.008066,1.008066
9,S10,1.020342,1.020342,1.020342,1.020342,1.020342,1.024674,1.147635,1.024674,1.024674,...,1.014523,1.014523,1.014523,1.014523,1.008714,1.008714,1.008714,1.008714,1.008714,1.008714


In [538]:
### IMPORT COUNTS AND SEPARATE BY CLASS AND PERIODS

file_path_counts = r"inputs/counts_by_hour_grouped_sorted.csv"
base_counts_df = pd.read_csv(
    file_path_counts,
    delimiter=',',
    encoding='utf-8',
    decimal='.',        # ← this tells pandas how to parse decimals
    thousands=',',       # ← this tells pandas how to parse thousands
    quotechar='"'
)

# --- Ajustar direcciones ---
base_counts_df["Direction"] = base_counts_df["Direction"].replace({"EB": "EB", "WB": "WB"})

# --- Crear columna Seg/Dir ---
base_counts_df["Seg/Dir"] = base_counts_df["Segment"].astype(str) + base_counts_df["Direction"]

# --- Función para procesar cada clase ---
def process_class(df_class):
    # Convertir a formato largo
    df_long = df_class.melt(
        id_vars=["Seg/Dir", "Segment", "Direction", "Class"],
        value_vars=[str(h) for h in range(24)],
        var_name="Hour",
        value_name="Volume"
    )
    
    # Mapear hora a periodo
    df_long["Hour"] = df_long["Hour"].astype(int)
    df_long["Period"] = df_long["Hour"].map(hour_to_period)
    
    # Agregar por Segment/Direction/Class/Period
    df_period = df_long.groupby(
        ["Seg/Dir", "Segment", "Direction", "Class", "Period"], as_index=False, sort=False
    ).agg({"Volume": "mean"}).round(0)
    
    # Pivot a formato ancho (periodos como columnas)
    period_order = df_period['Period'].unique()
    df_wide = df_period.pivot(
        index=["Seg/Dir", "Segment", "Direction", "Class"],
        columns="Period",
        values="Volume"
    )[period_order].reset_index()
    
    # Mantener solo Seg/Dir como índice
    df_proc = df_wide.drop(columns=["Class", "Direction", "Segment"]).set_index("Seg/Dir")
    
    return df_proc

# --- Separar por clases y procesar ---
dfs_by_class = {}
for cls in base_counts_df["Class"].unique():
    df_cls = base_counts_df[base_counts_df["Class"] == cls].copy()
    dfs_by_class[cls] = process_class(df_cls)


'''
Vehicle Classifications follow FHWA standards:
Lights: FHWA Classes 1-3 [Light Duty Vehicles]
Medium A: Classes 4-5 [Buses and Single Unit 2 axles trucks] 
Medium B: Class 6-7 [Single Unit 3 or 4 axles Trucks]
Heavy A: Classes 8-10 [Single Trailer 3 or more axles trucks]
Heavy B: Classes 11-13 [Combination Trucks Multitrailer Trucks]
'''

# --- Ejemplo de uso ---
df_lights = dfs_by_class["Lights"]
df_mediumA = dfs_by_class["Medium A"]
df_mediumB = dfs_by_class["Medium B"]
df_heavyA = dfs_by_class["Heavy A"]
df_heavyB = dfs_by_class["Heavy B"]


def extract_class(df, class_name):

    hour_cols = [str(i) for i in range(24)]
    df_class = df[df["Class"] == class_name].copy()
    df_class = df_class.set_index("Seg/Dir")[hour_cols]
    df_class.columns.name = "Period"
    return df_class

df_lights_hour    = extract_class(base_counts_df, "Lights")
df_mediumA_hour      = extract_class(base_counts_df, "Medium A")
df_mediumB_hour      = extract_class(base_counts_df, "Medium B")
df_heavyA_hour    = extract_class(base_counts_df, "Heavy A")
df_heavyB_hour    = extract_class(base_counts_df, "Heavy B")

df_heavyA_hour

Period,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
Seg/Dir,,,,,,,,,,,,,,,,,,,,,
S1SB,234.0,227.674413,253.0,233.0,285.300000,300.600000,268.200000,178.200000,327.488145,282.730552,...,388.000000,323.000000,387.800000,408.800000,460.600000,516.653787,339.0,267.30,244.80,219.600000
S1NB,215.0,249.000000,234.0,303.0,254.700000,262.800000,257.400000,266.083443,424.200000,606.200000,...,356.000000,295.000000,424.200000,401.800000,401.800000,429.800000,303.0,271.80,220.50,215.100000
S2SB,121.0,108.000000,104.0,97.0,122.369320,146.700000,153.900000,96.600000,105.000000,150.600000,...,171.900000,159.780233,144.572760,187.015594,222.705672,151.547242,136.8,109.80,103.50,93.600000
S2NB,95.0,110.000000,100.0,133.0,111.000000,123.300000,126.900000,141.400000,260.400000,253.400000,...,197.000000,155.000000,193.227922,164.742861,184.800000,163.957957,131.4,108.90,85.50,94.886869
S3SB,179.0,156.000000,183.0,177.0,216.840319,225.900000,193.500000,315.000000,422.800000,480.200000,...,306.000000,262.000000,292.600000,322.000000,351.400000,415.800000,228.6,188.10,179.10,167.400000
S3NB,157.5,160.500000,145.0,193.5,201.480261,171.900000,161.000000,218.400000,312.200000,494.900000,...,237.500000,222.500000,297.500000,270.200000,278.600000,265.300000,180.0,185.85,166.95,174.500000
S4SB,188.0,179.000000,173.0,198.0,254.000000,263.000000,234.000000,266.415975,344.811731,398.549517,...,279.900000,262.000000,310.800000,324.800000,389.200000,413.000000,239.0,223.00,218.00,196.000000
S4NB,174.0,194.000000,165.0,229.0,240.300000,223.200000,182.700000,208.021294,399.810158,519.360120,...,276.300000,255.600000,216.844466,246.256517,309.214904,216.852257,213.3,237.00,189.90,204.995488
S5SB,177.0,161.000000,178.0,193.0,261.000000,282.000000,222.212127,248.574330,350.585707,427.755500,...,316.800000,312.000000,372.400000,401.800000,456.400000,436.800000,262.0,221.00,222.00,194.000000


In [539]:
# --- Lista de periodos según tus columnas ---
period_cols = ["Night","AM-Early","AM-Peak","AM-Shoulder","MD","PM-Shoulder","PM-Peak","PM-Late"]

# Diccionario de dataframes por clase
class_dfs = {
    "Lights": df_lights,
    "Medium A": df_mediumA,
    "Medium B": df_mediumB,
    "Heavy A": df_heavyA,
    "Heavy B": df_heavyB
}

projected_long_by_class = {}

for cls_name, df_class in class_dfs.items():
    df = df_class.copy()
    
    # Resetear índice Seg/Dir y extraer Segment y Direction
    df = df.reset_index()
    df["Segment"] = df["Seg/Dir"].str.extract(r"(\d+)")[0]    # solo los números
    df["Direction"] = df["Seg/Dir"].str.extract(r"([A-Z]+)")[0]  # solo las letras
    df["Class"] = cls_name
    
    # Melt usando las columnas de periodos
    df_long = df.melt(
        id_vars=["Seg/Dir","Segment","Direction","Class"],
        value_vars=period_cols,
        var_name="Period",
        value_name="AADT "+str(df["Class"][0])
    )
    
    # Normalizar SegDir (opcional)
    df_long["SegDir"] = df_long["Seg/Dir"].str.strip().str.upper().str.lstrip("S")
    
    projected_long_by_class[cls_name] = df_long

# Ejemplo: ver Lights
projected_long_lights_df = projected_long_by_class["Lights"]
projected_long_mediumA_df = projected_long_by_class["Medium A"]
projected_long_mediumB_df = projected_long_by_class["Medium B"]
projected_long_heaviesA_df = projected_long_by_class["Heavy A"]
projected_long_heaviesB_df = projected_long_by_class["Heavy B"]
projected_long_lights_df


,Seg/Dir,Segment,Direction,Class,Period,AADT Lights,SegDir
0,S10NB,10,S,Lights,Night,1123.0,10NB
1,S10SB,10,S,Lights,Night,727.0,10SB
2,S1NB,1,S,Lights,Night,1397.0,1NB
3,S1SB,1,S,Lights,Night,1357.0,1SB
4,S2NB,2,S,Lights,Night,1245.0,2NB
...,...,...,...,...,...,...,...
155,S7SB,7,S,Lights,PM-Late,3042.0,7SB
156,S8NB,8,S,Lights,PM-Late,2092.0,8NB
157,S8SB,8,S,Lights,PM-Late,2336.0,8SB
158,S9NB,9,S,Lights,PM-Late,1948.0,9NB


In [540]:
rows = []

for year in years:
    for seg in seg_params.index:  # e.g., "1NB", "1SB", etc.
        seg_data = seg_params.loc[seg]
        # Extraer parte numérica y dirección
        seg_numeric = ''.join(filter(str.isdigit, seg))  # e.g., "10"
        direction = seg[len(seg_numeric):]       
        for p, hrs, peak, tag in period_template:
            rows.append({
                "Year": year,
                "SegDir": seg,
                "Segment": seg_numeric,        
                "Direction": direction,     
                "Period": p,
                "Hours/Day": hrs,
                "Peak": peak,
                "4Periods": tag,

                # Parámetros técnicos
                "Length": seg_data["Length"],
                "Speed GP": seg_data["Speed_GP"],
                "Capacity GP": seg_data["CapPerLane_GP"] * seg_data["Lanes_GP"],
                "Alpha GP": seg_data["Alpha_GP"],
                "Beta GP": seg_data["Beta_GP"],
                "Speed ML": seg_data["Speed_ML"],
                "Capacity ML": seg_data["CapPerLane_ML"] * seg_data["Lanes_ML"],
                "Alpha ML": seg_data["Alpha_ML"],
                "Beta ML": seg_data["Beta_ML"],
                "MinToll": 0.5,
                "MinCapture": 0
            })

# --- plantilla base ---
first_model_df = pd.DataFrame(rows)

# --- merge para todas las clases ---
for cls_name, df_proj in projected_long_by_class.items():
    proj_merge_df = df_proj[["SegDir", "Period", f"AADT {cls_name}"]].copy()
    proj_merge_df.rename(columns={"AADT": f"AADT {cls_name}"}, inplace=True)
    
    first_model_df = first_model_df.merge(
        proj_merge_df,
        on=["SegDir", "Period"],
        how="left"
    )

# --- 1. Reshape growths a formato largo ---
growths_long = base_growth_df.melt(
    id_vars="SegmentMapped",
    var_name="Year",
    value_name="AnnualGrowth"
).copy()
growths_long["Year"] = growths_long["Year"].astype(int)

# --- 2. Calcular crecimiento acumulado desde 2025 ---
# Ordenamos por año y aplicamos cumprod
growths_long = growths_long.sort_values(["SegmentMapped", "Year"])
growths_long["GrowthFactor"] = (growths_long["AnnualGrowth"]).groupby(growths_long["SegmentMapped"]).cumprod()

# Ahora GrowthFactor(y) = factor acumulado 2025→y

# --- 3. Preparar plantilla ---
fm = first_model_df.copy()
fm["Year"] = fm["Year"].astype(int)
fm["Segment"] = fm["Segment"].astype(str).str.replace(r"^S", "", regex=True)
fm["SegmentMapped"] = "S" + fm["Segment"].astype(str)

# --- 4. Merge GrowthFactor ---
fm = fm.merge(
    growths_long[["SegmentMapped", "Year", "GrowthFactor"]],
    on=["SegmentMapped", "Year"],
    how="left"
)

fm["GrowthFactor"] = fm["GrowthFactor"].fillna(1.0)

# --- 5. Aplicar GrowthFactor a todas las clases ---
for cls_name in projected_long_by_class.keys():
    col = f"AADT {cls_name}"
    if col in fm.columns:
        fm[col] = (fm[col].fillna(0) * fm["GrowthFactor"]).round(1)

# --- 6. Limpieza ---
fm = fm.drop(columns=["SegmentMapped"])   # opcional

first_model_df = fm

first_model_df


,Year,SegDir,Segment,Direction,Period,Hours/Day,Peak,4Periods,Length,Speed GP,...,Alpha ML,Beta ML,MinToll,MinCapture,AADT Lights,AADT Medium A,AADT Medium B,AADT Heavy A,AADT Heavy B,GrowthFactor
0,2025,1NB,1,NB,Night,7,OP,NT,1.3,55,...,1.6,6.3,0.5,0,1397.0,37.0,10.0,248.0,30.0,1.0
1,2025,1NB,1,NB,AM-Early,1,OP,AM,1.3,55,...,1.6,6.3,0.5,0,5278.0,151.0,79.0,257.0,12.0,1.0
2,2025,1NB,1,NB,AM-Peak,3,Peak,AM,1.3,55,...,1.6,6.3,0.5,0,5478.0,214.0,53.0,432.0,10.0,1.0
3,2025,1NB,1,NB,AM-Shoulder,1,OP,AM,1.3,55,...,1.6,6.3,0.5,0,5124.0,227.0,63.0,599.0,18.0,1.0
4,2025,1NB,1,NB,MD,5,OP,MD,1.3,55,...,1.6,6.3,0.5,0,3734.0,132.0,36.0,380.0,11.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,2025,10SB,10,SB,AM-Shoulder,1,OP,AM,4.5,70,...,1.6,6.3,0.5,0,2699.0,131.0,57.0,410.0,15.0,1.0
156,2025,10SB,10,SB,MD,5,OP,MD,4.5,70,...,1.6,6.3,0.5,0,3874.0,141.0,49.0,414.0,13.0,1.0
157,2025,10SB,10,SB,PM-Shoulder,1,OP,PM,4.5,70,...,1.6,6.3,0.5,0,3700.0,100.0,25.0,204.0,9.0,1.0
158,2025,10SB,10,SB,PM-Peak,3,Peak,PM,4.5,70,...,1.6,6.3,0.5,0,3844.0,84.0,16.0,303.0,14.0,1.0


In [541]:
first_model_df["Capacity GP"] = first_model_df.apply(
    lambda row: seg_params.loc[row["SegDir"], 'Cap_GP'],
    axis=1
)

first_model_df["B1"] = first_model_df.apply(
    lambda row: beta_vals[row["Year"]].loc[row["4Periods"], 1],
    axis=1
)

first_model_df["B2"] = first_model_df.apply(
    lambda row: beta_vals[row["Year"]].loc[row["4Periods"], 2],
    axis=1
)

# Here we load th value of the counts and we multiply the peak hour values by a constant
lights_w = 1

heavies_w = 3
heavies_w_toll = 3
heavies_w_vot = 3

medium_A_w = 3 # TBD: Maybe try 2.5 or 2.75 for every pce value
medium_A_w_toll = 3
medium_A_w_vot = 3

medium_B_w = 3
medium_B_w_toll = 3
medium_B_w_vot = 3

heavy_A_w = 3
heavy_A_w_toll = 5
heavy_A_w_vot = 3

heavy_B_w = 3
heavy_B_w_toll = 3
heavy_B_w_vot = 3


# 2. Compute TotalLights
first_model_df["TotalLights"] = first_model_df["AADT Lights"] 

first_model_df["TotalMediumA"] = first_model_df["AADT Medium A"]

first_model_df["TotalMediumB"] = first_model_df["AADT Medium B"]

first_model_df["TotalHeavyA"] = first_model_df["AADT Heavy A"]

first_model_df["TotalHeavyB"] = first_model_df["AADT Heavy B"]

first_model_df["TotalVeh"] = first_model_df.apply(
    lambda row: row["TotalLights"] + row["TotalMediumA"] + row["TotalMediumB"] + row["TotalHeavyA"] + row["TotalHeavyB"],
    axis=1
)

first_model_df["Corridor PCE pre-fix"] = first_model_df.apply(
    lambda row: row["TotalLights"] * lights_w + row["TotalMediumA"] * medium_A_w + row["TotalMediumB"] * medium_B_w + row["TotalHeavyA"] * heavy_A_w + row["TotalHeavyB"] * heavy_B_w,
    axis=1
)

first_model_df["Corridor PCE"] = first_model_df["Corridor PCE pre-fix"] # To anulate the suppression


first_model_df.loc[first_model_df['Period'].isin(['AM-Peak', 'PM-Peak']), 'TotalLights'] *= peak_factor

first_model_df["HOV3"] = first_model_df.apply(
    lambda row: row["TotalLights"] * hov_percentage.loc[row['Year']]['HOV percentage'],
    axis=1
)

first_model_df.loc[first_model_df['Period'].isin(['AM-Peak', 'PM-Peak']), 'TotalMediumA'] *= peak_factor
first_model_df.loc[first_model_df['Period'].isin(['AM-Peak', 'PM-Peak']), 'TotalMediumB'] *= peak_factor
first_model_df.loc[first_model_df['Period'].isin(['AM-Peak', 'PM-Peak']), 'TotalHeavyA'] *= peak_factor
first_model_df.loc[first_model_df['Period'].isin(['AM-Peak', 'PM-Peak']), 'TotalHeavyB'] *= peak_factor

first_model_df["TotalVeh"] = first_model_df.apply(
    lambda row: row["TotalLights"] + row["TotalMediumA"] + row["TotalMediumB"] + row["TotalHeavyA"] + row["TotalHeavyB"],
    axis=1
)

first_model_df["InScopeLights"] = first_model_df.apply(
    lambda row: (row["TotalLights"] - row["HOV3"]) * seg_params.loc[row["SegDir"], 'Inscope'],
    axis=1
)

first_model_df["InScopeMediumA"] = first_model_df.apply(
    lambda row: row["TotalMediumA"] * seg_params.loc[row["SegDir"], 'Inscope'],
    axis=1
)

first_model_df["InScopeMediumB"] = first_model_df.apply(
    lambda row: row["TotalMediumB"] * seg_params.loc[row["SegDir"], 'Inscope'],
    axis=1
)

first_model_df["InScopeHeavyA"] = first_model_df.apply(
    lambda row: row["TotalHeavyA"] * seg_params.loc[row["SegDir"], 'Inscope'],
    axis=1
)

first_model_df["InScopeHeavyB"] = first_model_df.apply(
    lambda row: row["TotalHeavyB"] * seg_params.loc[row["SegDir"], 'Inscope'],
    axis=1
)

first_model_df["InScopeVeh"] = first_model_df.apply(
    lambda row: row["TotalVeh"] * seg_params.loc[row["SegDir"], 'Inscope'],
    axis=1
)

# first_model_df.to_csv('model_test.csv')

In [542]:
def get_speed(row):

    max_VC = 1.2  # TBD: check if we need to change this value
    ETC_discount = 0.15


    gp_pce = (
        row["TotalLights"] * lights_w + 
        row["TotalMediumA"] * medium_A_w +
        row["TotalMediumB"] * medium_B_w +
        row["TotalHeavyA"] * heavy_A_w +
        row["TotalHeavyB"] * heavy_B_w
    )

    speedGP =  row["Speed GP"] / (1 + row["Alpha GP"] * ((gp_pce / row["Capacity GP"]) ** row["Beta GP"]))

    timeGP = 60 * row["Length"] / speedGP

    gp_vc = gp_pce / row["Capacity GP"]

    return pd.Series([speedGP, timeGP], index=["Speed GP","Time GP"])

In [543]:
first_model_df[["Speed GP Real", "Time GP"]] = first_model_df.apply(
    get_speed, axis=1, result_type='expand'
)

first_model_df.to_csv('model_test.csv')

In [ ]:
new_counts_file = 'new_counts_only_peaks'

def get_pce(row):

    measured_speed = measured_speeds.loc[row['Period']][row['SegDir']]

    gp_pce = (
        row["TotalLights"] * lights_w + 
        row["TotalMediumA"] * medium_A_w +
        row["TotalMediumB"] * medium_B_w +
        row["TotalHeavyA"] * heavy_A_w +
        row["TotalHeavyB"] * heavy_B_w
    )

    speed_rate = row['Speed GP'] / measured_speed

    if speed_rate < 1:

        measured_speed = row['Speed GP'] - 0.01

    gp_pce_aux = row["Capacity GP"] * ((row['Speed GP'] / (measured_speed * row['Alpha GP']) - 1 / row['Alpha GP']) ** (1 / row['Beta GP']))

    if row["Period"] == "PM-Peak" or row["Period"] == "AM-Peak" or row["Period"] == "AM-Shoulder" or row["Period"] == "PM-Shoulder":

        pce_factor = gp_pce_aux / gp_pce

        pce_factor = np.clip(pce_factor, a_min = 0.95, a_max = 1.05)
    
    else:
        pce_factor = gp_pce_aux / gp_pce

        pce_factor = np.clip(pce_factor, a_min = 0.95, a_max = 1)

    # pce_factor = gp_pce_aux / gp_pce

    # pce_factor = np.clip(pce_factor, a_min = 0.7, a_max = 1.3)

    return pce_factor

    

In [545]:
first_model_df["PCE Factor"] = first_model_df.apply(
    lambda row: get_pce(row),
    axis=1
)

# first_model_df.to_csv('model_test.csv')

In [ ]:
cap_factor = first_model_df.pivot(
    index=["Period"],
    columns="SegDir",
    values="PCE Factor"
)

cap_factor.to_csv('inputs/pce_factors.csv')

cap_factor

SegDir,10NB,10SB,1NB,1SB,2NB,2SB,3NB,3SB,4NB,4SB,5NB,5SB,6NB,6SB,7NB,7SB,8NB,8SB,9NB,9SB
Period,,,,,,,,,,,,,,,,,,,,
AM-Early,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,1.000000,0.900000,0.900000,1.000000,0.900000,1.000000,0.900000,1.000000,1.000000,1.000000,0.983181,0.900000,0.979691,0.999902
AM-Peak,1.005464,1.004793,1.106380,0.600000,1.150969,0.622894,1.400000,1.116935,1.000563,1.000830,0.998042,1.002728,0.997170,1.002596,1.131692,1.000528,1.190229,1.003981,1.001948,1.001204
AM-Shoulder,0.999372,1.000267,1.089292,0.999850,1.148364,0.600000,1.343771,1.013225,0.999923,1.000461,0.999892,1.000327,1.000140,1.000191,0.999837,0.999710,0.999956,0.999237,0.999320,0.999221
MD,0.927248,0.900000,1.000000,1.000000,1.000000,0.900000,1.000000,1.000000,0.900000,1.000000,0.980433,1.000000,0.963490,0.900000,0.966518,0.900000,0.999833,0.952625,1.000000,0.906253
Night,1.000000,1.000000,0.900000,0.900000,1.000000,1.000000,1.000000,0.933882,0.900000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
PM-Late,1.000000,1.000000,1.000000,1.000000,0.900000,0.900000,0.900000,0.900000,0.900000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
PM-Peak,1.005748,1.008373,1.216745,1.147372,1.081101,1.001031,1.400000,1.233229,1.084125,1.400000,1.015237,1.049020,1.014408,0.998926,1.023656,0.999943,1.038075,1.009896,1.048504,1.000724
PM-Shoulder,1.000055,0.999373,1.349971,1.324666,1.000026,0.999985,1.400000,1.400000,0.999648,1.400000,0.999484,1.076799,0.999484,1.000487,1.000181,1.000261,1.000308,1.000055,1.000243,1.000108


In [547]:
# period_map = {0: "Night", 1:"Night", ...}
period_map_series = pd.Series(hour_to_period)

dropped_columns = ['Segment', 'Direction', 'Class', 'AADT']

df_counts = base_counts_df.drop(columns=dropped_columns).groupby(['Seg/Dir']).sum()

df_counts.index = df_counts.index.str.lstrip('S')
       # your traffic counts dataframe
df_factors = cap_factor        # your period × segdir dataframe

hours = list(range(24))
output = df_counts.copy()

df_counts

for h in hours:
    period = period_map_series[h]           # "AM-Early", "MD", etc.
    factors = df_factors.loc[period]        # row of segdir factors

    output[str(h)] = df_counts.apply(
        lambda row: row[str(h)] * factors[row.name],
        axis=1
    )

# df_counts.to_csv('base_total_counts.csv')
# output.to_csv(f'{new_counts_file}.csv')

In [548]:
first_model_df["TotalLights"] = first_model_df.apply(
    lambda row: row["TotalLights"] * cap_factor.loc[row["Period"], row["SegDir"]],
    axis=1
)

first_model_df["TotalMediumA"] = first_model_df.apply(
    lambda row: row["TotalMediumA"] * cap_factor.loc[row["Period"], row["SegDir"]],
    axis=1
)

first_model_df["TotalMediumB"] = first_model_df.apply(
    lambda row: row["TotalMediumB"] * cap_factor.loc[row["Period"], row["SegDir"]],
    axis=1
)

first_model_df["TotalHeavyA"] = first_model_df.apply(
    lambda row: row["TotalHeavyA"] * cap_factor.loc[row["Period"], row["SegDir"]],
    axis=1
)

first_model_df["TotalHeavyB"] = first_model_df.apply(
    lambda row: row["TotalHeavyB"] * cap_factor.loc[row["Period"], row["SegDir"]],
    axis=1
)

In [549]:
def get_capacity(row):

    measured_speed = measured_speeds.loc[row['Period']][row['SegDir']]

    gp_pce = (
        row["TotalLights"] * lights_w + 
        row["TotalMediumA"] * medium_A_w +
        row["TotalMediumB"] * medium_B_w +
        row["TotalHeavyA"] * heavy_A_w +
        row["TotalHeavyB"] * heavy_B_w
    )

    speed_rate = row['Speed GP'] / measured_speed

    if speed_rate < 1:

        measured_speed = row['Speed GP'] - 0.01

    capacity = gp_pce / ((row['Speed GP'] / (measured_speed * row['Alpha GP']) - 1 / row['Alpha GP']) ** (1 / row['Beta GP']))

    capacity_factor = capacity / row["Capacity GP"]

    return capacity_factor

In [ ]:
first_model_df["Capacity Factor"] = first_model_df.apply(
    lambda row: get_capacity(row),
    axis=1
)

cap_factor = first_model_df.pivot(
    index=["Period"],
    columns="SegDir",
    values="Capacity Factor"
)

cap_factor.to_csv('inputs/capacity_factors.csv')

cap_factor

SegDir,10NB,10SB,1NB,1SB,2NB,2SB,3NB,3SB,4NB,4SB,5NB,5SB,6NB,6SB,7NB,7SB,8NB,8SB,9NB,9SB
Period,,,,,,,,,,,,,,,,,,,,
AM-Early,1.551513,1.181301,1.381588,2.080574,2.870833,1.907587,0.953073,2.441673,1.419517,0.912481,1.178351,0.999870,1.065160,0.999376,0.999994,0.999986,1.000000,1.013142,1.000000,1.000000
AM-Peak,1.000000,1.000000,1.000000,1.179402,1.000000,1.000000,0.972991,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
AM-Shoulder,1.000000,1.000000,1.000000,1.000000,1.000000,1.196102,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
MD,1.000000,1.309905,0.680234,0.916707,0.878314,3.046656,0.651942,0.726926,1.134325,0.870613,1.000000,0.973710,1.000000,1.002172,1.000000,1.019467,1.000000,1.000000,0.980208,1.000000
Night,0.402524,0.344219,1.121109,1.113547,0.894871,0.837103,0.852858,1.000000,1.062373,0.372992,0.489647,0.395506,0.408933,0.374743,0.389767,0.363897,0.414749,0.403431,0.374844,0.308923
PM-Late,0.591799,0.716519,0.838231,0.850829,1.511985,1.611712,1.162702,1.792735,1.372921,0.621295,0.669994,0.632106,0.543515,0.652107,0.518949,0.660895,0.525885,0.591934,0.517884,0.590669
PM-Peak,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.916257,1.000000,1.000000,0.929849,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
PM-Shoulder,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.867378,0.984381,1.000000,0.836324,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [551]:
period_order = [
    "Night",
    "AM-Early",
    "AM-Peak",
    "AM-Shoulder",
    "MD",
    "PM-Shoulder",
    "PM-Peak",
    "PM-Late"
]

first_model_df["Total Corridor"] = first_model_df.apply(
    lambda row: row["Corridor PCE"] * row["Hours/Day"],
    axis=1
)


first_model_df["Period"] = pd.Categorical(
    first_model_df["Period"],
    categories=period_order,
    ordered=True
)

corridor_pce = first_model_df.pivot(
    index=["Period"],
    columns="SegDir",
    values="Total Corridor"
)

# corridor_pce.to_csv('corridor_vals.csv')